In [ ]:
import pandas as pd 
import numpy as np
import json
import re
from string import punctuation
from glob import glob
import tensorflow as tf
import collections
import bisect
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
## The path of files may need to be modified
label_map = {'CN': 0, 'AD': 1}
"""
filenames = tf.io.gfile.glob("/content/gdrive/MyDrive/speech/test/transcription/*.json")
filenames = tf.random.shuffle(filenames).numpy()
segmentations = tf.io.gfile.glob(
    "/content/gdrive/MyDrive/speech/test/segmentation/*.csv"
)
test_labels_csv = pd.read_csv("/content/gdrive/MyDrive/speech/test/test_results_task1_groundtruth.csv")

"""
filenames = tf.io.gfile.glob("/content/gdrive/MyDrive/DementiaBank.20210603/ADReSSo2021/diagnosis/train/transcription/*.json")
filenames = tf.random.shuffle(filenames).numpy()
segmentations = tf.io.gfile.glob("/content/gdrive/MyDrive/DementiaBank.20210603/ADReSSo2021/diagnosis/train/segmentation/*/*.csv")

In [ ]:
all_time_line = collections.defaultdict(list)
labels = collections.defaultdict(str)
ids = []
for segmentation in segmentations:
  seg_file = pd.read_csv(segmentation)
  seg_file = seg_file[seg_file["speaker"] == "PAR"]
  file_id = segmentation.split("/")[-1].split(".")[0]
  label = segmentation.split("/")[-2]
  ids.append(file_id)
  labels[file_id] = label
  for idx, r in seg_file.iterrows():
    speaker = r['speaker']
    start_time = r['begin']
    end_time = r['end']

    if type(start_time) == str:
        start_time = float(start_time.strip(' ').strip('"'))
      
    if type(end_time) == str:
        end_time = float(end_time.strip(' ').strip('"'))

    # convert ms to seconds
    start_time = start_time / 1000.0
    end_time = end_time / 1000.0

    all_time_line[file_id].append((start_time, end_time))
  
  idx = 0
  while(1):
    try:
      if all_time_line[file_id][idx][1] >= all_time_line[file_id][idx+1][0]:
        temp = (all_time_line[file_id][idx][0], all_time_line[file_id][idx+1][1])
        all_time_line[file_id].pop(idx)
        all_time_line[file_id].pop(idx)
        all_time_line[file_id].insert(idx, temp)
      else:
        idx += 1
    except:
      break

In [ ]:
def is_patient_utterance(speech_timeline, time_point):
    eps = 10e-4
    search_point = (time_point[0] + eps, time_point[1])
    idx = bisect.bisect_left(speech_timeline, search_point)
    # print(speech_timeline,search_point, idx)
    if idx == 0:
        return False
    return speech_timeline[idx - 1][0] <= time_point[0] <= time_point[1] <= speech_timeline[idx - 1][1]

In [ ]:
def get_interval_from_file(file):
  segments = file["results"]['speaker_labels']['segments']
  start_time = []
  end_time = []
  for segment in segments:
    # print(segment['items'])
    for word in segment['items']:
      # print(word)
      start_time.append(float(word['start_time']))
      end_time.append(float(word['end_time']))
  interval = np.append(np.array([0]), np.array(start_time[1:]) - np.array(end_time[:-1]))
  return start_time, end_time, interval

In [ ]:
def remove_original_punctuation(file):
  for transcript in file["results"]["transcripts"]:
    # print(transcript['transcript'])
    text = re.sub('[{}]'.format(punctuation),'',transcript['transcript'])
    text = text.split(" ")
  return text

In [ ]:
def delete_other_speaker(all_time_line, file_id, start_time, end_time):
  time_line = all_time_line[file_id]
  del_index = []
  idx = 0
  for time_point in zip(start_time, end_time):
    if not is_patient_utterance(time_line, time_point):
      del_index.append(idx)
    idx += 1
  return del_index

In [ ]:
ids = []
labels = []
texts = []
result = []
for filename in filenames:
  with open(filename) as f:
    file = json.load(f)
  filename = filename.decode("utf-8")
  file_id = filename.split(".wav")[0].split("-")[-1]
  start_time, end_time, interval = get_interval_from_file(file)
  del_index = delete_other_speaker(all_time_line, file_id, start_time, end_time)
  text = remove_original_punctuation(file)
  text = np.delete(text, del_index)
  interval = np.delete(interval, del_index)
  interval = np.rint(interval)
  interval = interval.astype(int)
  ids.append(file_id)
  """
  l = test_labels_csv.loc[test_labels_csv['ID'] == file_id].iloc[0]["Dx"]
  if l == "Control":
    labels.append(0)
  else:
    labels.append(1)
  """
  labels.append(label_map[filename.split(".wav")[0].split("-")[-2]])
  texts.append(' '.join(str(i) for i in text))
  result.append({"name": file_id,"label": labels[-1], "content": ' '.join(str(i) for i in text),"pause": ' '.join(str(i) for i in interval)})

In [ ]:
df = pd.DataFrame(data=result)

In [ ]:
df

,name,label,content,pause
0,adrso202,1,Mhm Can start others grind jesus and the sink ...,1 2 0 0 0 0 2 0 0 0 0 0 0 1 2 2 0 0 0 0 0 2 0 ...
1,adrso253,1,Well my boys trying to get some cookies and th...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,adrso245,1,Im very climbed up on the same your foot jar w...,2 0 0 0 0 0 0 0 0 3 2 0 0 0 0 0 1 0 0 0 0 0 0 ...
3,adrso286,0,Uh huh Well one thing this boys on the stool g...,0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 ...
4,adrso036,1,All the Whats going on in this picture Was tha...,0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 2 0 ...
...,...,...,...,...
161,adrso123,1,Mhm Well the sink is running over Shes drying ...,1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 2 ...
162,adrso198,1,Thank you mm Helaine Helen and our Children Tr...,1 0 1 0 1 0 0 0 2 0 0 0 2 0 0 2 0 3 1 0 0 2 1 ...
163,adrso134,1,Yeah Yeah Voice touching A K Yes Girls reachin...,2 2 2 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
164,adrso165,0,Yeah Girls reaching up The boys is taking cook...,2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [ ]:
df.to_csv("train.csv")